In [ ]:
import os

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

## Need to change code 

### code from mmseg/model/segmentors/encoder_decoder.py

### change code line 282 from seg_pred = seg_logit.argmax(dim=1) to seg_pred = seg_logit 

In [ ]:
# config file 들고오기
cfg = Config.fromfile('./configs/_custom_/models/upernet_swin_l.py')
root='../input/mmseg/test/'
epoch = 'latest'

# dataset config 수정
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = '../output/mmseg/upernet_swin_l_alldata'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [ ]:
output_mmseg = single_gpu_test(model, data_loader)
print("done")

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import yaml
import ttach as tta

from tools import test
from model.custom_encoder import register_encoder
import segmentation_models_pytorch as smp

@torch.no_grad()
def inference(cfg, device):
    # custom encoder smp에 등록
    register_encoder()

    # TTA transform
    tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Scale(scales=[0.5, 0.75, 1.0, 1.25, 1.5])
    ])
   
    model_path =  f"{cfg['saved_dir']}/{cfg['exp_name']}/{cfg['exp_name']}.pt"                                    
    
    # model 불러오기
    model = smp.__dict__[cfg['model']['name']]
    model = model(encoder_weights=None, **cfg['model']['params'])
    # best model 불러오기
    checkpoint = torch.load(model_path, map_location=device)
    state_dict = checkpoint.state_dict()
    model.load_state_dict(state_dict)

    model = model.to(device)
    # 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
    model.eval()

    tta_model = tta.SegmentationTTAWrapper(model, tta_transforms, merge_mode='mean')

    # test set에 대한 prediction
    file_names, preds = test(tta_model, data_loader, device)

    return preds

In [ ]:

yaml_path = "/opt/ml/segmentation/baseline_code/saved/PAN_Swin_Alldata/PAN_Swin_Alldata.yaml"

with open(yaml_path) as f:
    cfg = yaml.safe_load(f)

device = "cuda" if torch.cuda.is_available() else "cpu"

output_base = inference(cfg, device)

In [ ]:
output = []

for predict1, predict2 in zip(output_mmseg, output_base):
  predict = predict1 + predict2
  predict = predict.argmax(axis=0)
  output.append(predict)

output = np.array(output)
print(output.shape)

In [ ]:
# sample_submisson.csv 열기
submission = pd.read_csv('../output/sample_submission.csv', index_col=None)
json_dir = os.path.join("../input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join('../output/', f'ensemble_{epoch}.csv'), index=False)